# Import 

In [1]:
import re 
import numpy as np 
import pandas as pd
from argparse import Namespaces
import pickle 
import torch 

import sys
import os

current_path = notebook_dir = os.getcwd()
working_dir = os.path.abspath(os.path.join(current_path, '..','..'))
if working_dir not in sys.path:
    sys.path.insert(0, working_dir)
    
from high_level_DL_method import load_model,load_optimizer_and_scheduler
from trainer import Trainer
from examples.train_and_visu_non_recurrent import get_multi_ds
from utils.metrics import evaluate_metrics
from jupyter_ipynb.NetMob_training_analysis.utils import load_model_args,load_K_fold_datasets,load_trained_model,get_masked_previous_preds_true,SAVE_FOLDER_PATH,L_Apps,get_df_results,get_metrics_from_test
from jupyter_ipynb.NetMob_training_analysis.plotting import plot_boxplot_on_metric

ImportError: cannot import name 'Namespaces' from 'argparse' (/root/anaconda3/envs/pytorch-2.0.1/lib/python3.10/argparse.py)

In [2]:
folder_name = 'subway_in_only'
trial_id = 'subway_in_STGCN_MSELoss_2025_01_20_14_27_20569'
model_args = load_model_args(folder_name)
df1 = get_df_results(trial_id,model_args,L_Apps =[],name_id= 'subway_in_only')

folder_name = 'subway_in_subway_in'
trial_id = 'subway_in_subway_out_STGCN_MSELoss_2025_02_19_00_05_19271'
model_args = load_model_args(folder_name)
df2 = get_df_results(trial_id,model_args,L_Apps =[],name_id= 'subway_in')


df_app = pd.read_csv('/home/rrochas/prediction-validation/Metrics_on_each_fold_and_apps.csv',index_col = 0)

df = pd.concat([df_app,df2,df1])
df['fold_str'] = df['fold'].apply(lambda x: str(x))
df['fold'] = df['fold'].apply(lambda x: int(x))

NameError: name 'load_model_args' is not defined

## Load Metrics on each fold:
### Un csv à été suavegardé et qui contient toute les valeurs des métriques.

## Idem pour subway-in/subway-out. Cette fois ci pas de 'NetMob'. juste plus de trial:

In [2]:
## Load NetMob Metrics : 
folder_name = 're_validation_epsilon100'
trial_id = 'subway_in_subway_out_STGCN_MSELoss_2025_02_19_00_05_19271NETMOB_eps100'
model_args = load_model_args(folder_name)
df1 = get_df_results(trial_id,model_args,L_Apps)

folder_name = 're_validation'
trial_id = 'subway_in_subway_out_STGCN_MSELoss_2025_02_19_00_05_19271NETMOB_eps100'
model_args = load_model_args(folder_name)
df2 = get_df_results(trial_id,model_args,L_Apps)

df_netmob = pd.concat([df1,df2])
df_netmob.head()


## Load Subway-in/Subway-out metrics: 
folder_name = 'tmps'
trial_id = 'subway_in_subway_out_STGCN_MSELoss_2025_02_19_00_05_19271'
model_args = load_model_args(folder_name)

df_subway_out = pd.DataFrame(columns = ['mse','mae','mape','mase','fold','trial_num'])
pattern = re.compile(rf"{trial_id}_\d_")
best_model_names = [name for name in model_args['model'].keys() if pattern.search(name)]

for selected_model_name in best_model_names:
    model_metrics = model_args['model'][selected_model_name]['performance']['test_metrics']
    trial_id = selected_model_name.split('_')[-2]
    if trial_id == 'fcomplete':
        trial_id = selected_model_name.split('_')[-3]
    fold_k = selected_model_name[-1]
    if fold_k == 't':
        fold_k = 4

    df_subway_out.loc[len(df_subway_out)] = [model_metrics['mse'],model_metrics['mae'],model_metrics['mape'],model_metrics['mase'],fold_k,trial_id]
df_subway_out['id'] = 'subway-out'
df_subway_out.head()


df = pd.concat([df_netmob,df_subway_out])
df.head()

,mse,mae,mape,mase,fold,id,trial_num
0,1365.452881,21.565752,43.349380,NaN,0,Apple_Video,2
1,1329.773315,21.960958,36.967972,NaN,1,Apple_Video,2
2,1264.973877,20.838892,30.041185,NaN,2,Apple_Video,2
3,1233.327148,20.747341,29.113092,NaN,3,Apple_Video,2
4,1296.285156,20.662750,28.951359,NaN,4,Apple_Video,2


## Plotting:

In [38]:
if True: 
    #save_path=f"MSE_distribution_per_app_and_per_fold.html"
    save_path = f"MASE_distribution_per_app_and_per_fold.html"
    metric_i = 'mase'
    plot_boxplot_on_metric(df,metric_i,save_path)
if True:
    save_path = f"MAE_distribution_per_app_and_per_fold.html"
    metric_i = 'mae'
    plot_boxplot_on_metric(df,metric_i,save_path)

    save_path = f"MSE_distribution_per_app_and_per_fold.html"
    metric_i = 'mse'
    plot_boxplot_on_metric(df,metric_i,save_path)
if False:
    agg_metrics = df.groupby(['fold']).agg(mse=('mse','mean'),mae=('mae','mean'),mase=('mase','mean'),mape=('mape','mean'))
    
    for metric_i in ['mase','mae','mse']:
        df['fold_str'] = df['fold'].apply(lambda x : f"{x}. mean {metric_i.upper()}: {agg_metrics.loc[x,metric_i]}")
        plot_boxplot_on_metric(df,metric_i,f"{metric_i.upper()}_distribution_on_SUBWAY_OUT_per_fold.html")

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

In [ ]:
# Init 

for num_folder,folder_name in enumerate(['re_validation_epsilon100','re_validation']):
    
    model_args = load_model_args(folder_name)
    trial_id = 'subway_in_subway_out_STGCN_MSELoss_2025_02_19_00_05_19271NETMOB_eps100'

    globals()[f"pandas_total_results{num_folder}"] = pd.DataFrame()
    for app_ind,app in enumerate(L_Apps):
        print('avancement: ',(app_ind+1)/len(L_Apps))
        best_model_names = [name for name in model_args['model'].keys() if (f"{trial_id}_{app}_f" in name) or (f"{trial_id}_{app}_1_f" in name) or (f"{trial_id}_{app}_2_f" in name)]
        if len(best_model_names)>5:
            best_model_names=best_model_names[:5]

        ## Load datasets: 
        # Load Args 
        # Use information from first fold to get the inputs (which app, which train/valid/test prop...):
        args_0 = model_args['model'][best_model_names[0]]['args']
        args_0 = Namespace(**args_0)
        args_with_contextual,K_subway_ds = get_multi_ds(args_0.model_name, args_0.dataset_names,args_0.dataset_for_coverage,args_init = args_0,fold_to_evaluate = np.arange(args_0.K_fold))
        # ...
        ## ====

        # With the already load ds, load now the trained model. And ds is ready for the evaluation:
        for k_fold,model_fold_i in enumerate(best_model_names):
            # Load trained param: 
            ds = K_subway_ds[k_fold]
            #selected_model_path = f"{current_path}/{save_path}/{model_fold_i}.pkl"

            # Model :
            trainer = load_trained_model(model_args,ds,model_fold_i,folder_name)

            # get metrics : 
            dic_pred_metrics = get_metrics_from_test(trainer,ds,metric_list = ['mse','mae','mape','mase'])

            pd_local_results = pd.DataFrame(dict(mse = [dic_pred_metrics['mse']],mase = [dic_pred_metrics['mase']],mae = [dic_pred_metrics['mae']], mape = [dic_pred_metrics['mape']],
                                                fold = [k_fold], id = [app], trial_num = [2] if 'eps' in folder_name else [1]))
            globals()[f"pandas_total_results{num_folder}"] = pd.concat([ globals()[f"pandas_total_results{num_folder}"],pd_local_results])
       
csv_with_mase = pd.concat([ globals()[f"pandas_total_results0"], globals()[f"pandas_total_results1"]])



## Ajout des csv_with_mase sur les df existant : 
csv_with_mase['fold'] = csv_with_mase['fold'].apply(lambda x: int(x)) 
csv_with_mase['trial_num'] = csv_with_mase['trial_num'].apply(lambda x: int(x)) 
df['fold'] = df['fold'].apply(lambda x: int(x)) 
df['trial_num'] = df['trial_num'].apply(lambda x: int(x)) 

def f_to_apply(row,csv_with_mase):
    try:
        mase = csv_with_mase[(csv_with_mase['id'] == row['id']) &
                                    (csv_with_mase['fold'] == row['fold']) &
                                    (csv_with_mase['trial_num'] == row['trial_num']) 
                                    ]['mase'].iloc[0]
        return mase
    except:
        return row['mase']
 

df['mase'] = df.apply(lambda row : f_to_apply(row,csv_with_mase),axis = 1)


avancement:  0.014705882352941176
>>>>Model: STGCN; K_fold = 6; Loss function: MSE 
Invalid dates within this fold: 776

Init Dataset: 'torch.Size([7392, 40]) with 295680 Total nb of elements and 0 Nan values
nb subway_in invalid dates:  776
Netmob_T.size():  torch.Size([7392, 40])
vision_input_type POIs
vision_model_name None
2821 train samples had been added thank to Data Augmentation
calendar data augmented by dupplication but not modified
Train/Target size:  torch.Size([5642, 40, 7]) torch.Size([5642, 40, 1])
Init U/Utarget size: torch.Size([4702, 40, 7])/torch.Size([4702, 40, 1]) Train/Valid/Test 5642 940 940

 ===== ERROR WITH prefetch_factor====  
ValueError: prefetch_factor option could only be specified in multiprocessing.let num_workers > 0 to enable multiprocessing
Invalid dates within this fold: 481
Considered Spatial-Unit:  Index(['AMP', 'BEL', 'BRO', 'COR', 'CUI', 'CUS', 'FLA', 'GOR', 'BLA', 'GRA',
       'GUI', 'GIL', 'HEN', 'HOT', 'LAE', 'MAS', 'MER', 'LUM', 'PRY', 'PER

,mse,mase,mae,mape,fold,id,trial_num
0,1365.452881,0.759169,21.565752,43.349380,0,Apple_Video,2
0,1329.773315,0.727521,21.960958,36.967972,1,Apple_Video,2
0,1264.973877,0.659811,20.838892,30.041185,2,Apple_Video,2
0,1233.327148,0.645517,20.747341,29.113092,3,Apple_Video,2
0,1296.285156,0.645276,20.662750,28.951359,4,Apple_Video,2
...,...,...,...,...,...,...,...
0,1305.429443,0.748193,21.249725,39.232002,0,Fortnite,1
0,1296.882202,0.719855,21.731592,36.273453,1,Fortnite,1
0,1374.309082,0.674467,21.300262,28.725454,2,Fortnite,1
0,1232.037109,0.650366,20.894806,30.711609,3,Fortnite,1


In [39]:
import pandas as pd
import numpy as np

from bokeh.plotting import figure, show
from bokeh.models import (ColumnDataSource, HoverTool, FactorRange,
                          Toggle, CustomJS, Legend, LegendItem)
from bokeh.layouts import layout, column
from bokeh.transform import factor_cmap
from bokeh.palettes import Category10
from bokeh.io import output_notebook, output_file, save, reset_output
from bokeh.document import Document
from bokeh.embed import file_html
from bokeh.resources import CDN
from bokeh.models import Span

def return_stats(df,metric):
    # Calculs min, q1, median, q3, max par ID (distribution sur folds)
    grp = df.groupby("id")[metric]
    q1 = grp.quantile(0.25)
    q2 = grp.quantile(0.50)
    q3 = grp.quantile(0.75)
    mn = grp.min()
    mx = grp.max()

    stats = pd.DataFrame({
        "id": q1.index,
        "min_v": mn.values,
        "q1": q1.values,
        "median_v": q2.values,
        "q3": q3.values,
        "max_v": mx.values
    })
    return stats

# -------------------------------------------------------------------------
# 1) Sous-fonction pour créer le boxplot "classique" (fold/couleurs)
#    Inspiré directement de ton code existant
# -------------------------------------------------------------------------
def create_boxplot_by_fold(p, df, metric):
    """
    Crée le boxplot pour chaque 'id', couleurs par 'fold'.
    Retourne la liste des renderers pour pouvoir toggler leur visibilité.
    """
    # S'assure d'avoir fold_str (couleur par fold)
    if 'fold_str' not in df.columns:
        df["fold_str"] = df["fold"].astype(str)

    # get stats
    stats = return_stats(df,metric)

    # Sources
    source_box = ColumnDataSource(stats)
    source_points = ColumnDataSource(df)

    # Boxplot
    r1 = p.segment("id", "max_v", "id", "q3", source=source_box, line_color='black')
    r2 = p.segment("id", "min_v", "id", "q1", source=source_box, line_color='black')
    r3 = p.vbar("id", 0.2, "median_v", "q3",
                source=source_box,
                fill_color='grey', fill_alpha=0.3, line_color='black')
    r4 = p.vbar("id", 0.2, "q1", "median_v",
                source=source_box,
                fill_color='grey', fill_alpha=0.3, line_color='black')

    # Points (couleurs par fold)
    folds_unique = sorted(df["fold_str"].unique())
    palette = Category10[len(folds_unique)] if len(folds_unique) <= 10 else None

    circle_renderers = []
    folds_unique = df["fold_str"].unique()
    for i, fold_val in enumerate(folds_unique):
        subset = df[df["fold_str"] == fold_val]
        source_subset = ColumnDataSource(subset)
        r_circle = p.scatter(
            x="id", y=metric,
            source=source_subset,
            size=7,
            line_color="black",
            fill_color=Category10[len(folds_unique)][i],
            legend_label=str(fold_val)
        )
        circle_renderers.append(r_circle)
    p.legend.click_policy = "hide"

    return [r1, r2, r3, r4] + circle_renderers


# -------------------------------------------------------------------------
# 2) Sous-fonction pour créer un boxplot basé sur la distribution "id x trial"
#    On agrège par (id, trial_num)
# -------------------------------------------------------------------------
def create_boxplot_by_trial(p, df, metric):
    """
    Crée un boxplot où l'on considère la distribution des trials pour chaque ID.
    On ignore la dimension "fold" ici (on pourrait garder 'fold_str' si besoin).
    """
    # Agg par (id, trial_num)
    df2 = df.drop(columns=['fold_str','fold'], errors='ignore').copy()
    df2 = df2.groupby(['id', 'trial_num'], as_index=False).mean()
    
    stats = return_stats(df2,metric)
    
    source_box = ColumnDataSource(stats)
    source_points = ColumnDataSource(df2)

    r1 = p.segment("id", "max_v", "id", "q3", source=source_box, line_color='black')
    r2 = p.segment("id", "min_v", "id", "q1", source=source_box, line_color='black')
    r3 = p.vbar("id", 0.2, "median_v", "q3",
                source=source_box,
                fill_color='grey', line_color='black', fill_alpha=0.3)
    r4 = p.vbar("id", 0.2, "q1", "median_v",
                source=source_box,
                fill_color='grey', line_color='black', fill_alpha=0.3)

    # Petits cercles pour chaque (id, trial_num)
    r5 = p.scatter(
        x="id", y=metric,
        source=source_points,
        size=5, color='blue', alpha=0.6
    )

    return [r1, r2, r3, r4, r5]


# -------------------------------------------------------------------------
# 3) Sous-fonction pour tracer la moyenne globale (par ID) sous forme de croix
# -------------------------------------------------------------------------
def create_mean_cross_by_id(p, df, metric):
    """
    Affiche une marque (croix) à la moyenne globale (tous folds, trials confondus),
    groupée par 'id'.
    """
    df3 = df.drop(columns=['fold_str','fold','trial_num'], errors='ignore').copy()
    df3 = df3.groupby(['id'], as_index=False).mean()

    source_mean = ColumnDataSource(df3)

    # On va utiliser un marker 'x' pour tracer la moyenne
    # (taille un peu plus grande, couleur rouge, etc.)
    r = p.scatter(marker = 'cross',
        x="id", y=metric,
        size=15, line_color="red", fill_color=None,
        source=source_mean
    )
    return [r]

#
#
#
def add_v_lines(p):
    for cat in ["subway_in", "subway_in_only"]:
        if cat in p.x_range.factors:
            # Récupère l'index numérique associé à cette catégorie
            idx = p.x_range.factors.index(cat)
            # Crée un Span infini vertical (dimension="height") au niveau de 'idx'
            vline = Span(
                location=idx,
                dimension="height",          # une ligne verticale
                line_color='#4292c6',
                line_dash="dashed",
                line_width=2
            )
            p.add_layout(vline)

# -------------------------------------------------------------------------
# 4) Fonction principale qui assemble tout et crée les toggles
# -------------------------------------------------------------------------
def plot_boxplot_on_metric_with_layers(df, metric='mse', save_path=None):
    """
    Crée un unique plot Bokeh avec 3 couches (fold_boxplot, trial_boxplot, mean_cross).
    Chaque couche est toggle-able.
    """
    # Ordonner l'axe x par la moyenne (optionnel)
    sdf = df.groupby("id")[metric].mean().sort_values()
    sdf_ids = sdf.index.tolist()

    p = figure(
        x_range=sdf_ids,
        width=1200,
        height=500,
        title=f"{metric.upper()} distribution"
    )
    p.xaxis.axis_label = "App"
    p.yaxis.axis_label = metric.upper()
    p.xaxis.major_label_orientation = np.pi/2

    # 4a) Ajout des 3 couches
    functions = [create_boxplot_by_fold,create_boxplot_by_trial,create_mean_cross_by_id]
    labels = ["Fold","trial","mean"]
    full_names = ["Fold distribution","Trial distribution","Mean Cross"]

    for f,label,full_name in zip(functions,labels,full_names):
        renderers = f(p, df, metric)
        globals()[f"toggle_{label}"] = Toggle(label=full_name, active=True)
        globals()[f"toggle_{label}"].js_on_change("active", CustomJS(args=dict(rend=renderers),
            code="""
            rend.forEach(r => r.visible = cb_obj.active);
            """))
    layout_obj = column(p, *[globals()[f"toggle_{label}"] for label in labels])
    
    add_v_lines(p)

    # 4d) Affichage notebook
    output_notebook()
    show(layout_obj)

    # 4e) Sauvegarde optionnelle
    if save_path:
        reset_output()
        output_file(save_path)
        save(layout_obj)
        reset_output()

for metric in ['mse','mae','mape','mase']:
    plot_boxplot_on_metric_with_layers(df, metric=metric, save_path=f"{metric.upper()}_distribution_per_app_and_per_fold.html")


Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

In [32]:
df_copy['mase'].max()

0.9509536623954772

In [27]:
df_copy = df.copy()
df_copy.drop(columns = ['fold_str','fold']).groupby(['id','trial_num']).mean()

mse        mae       mape      mase
id                  trial_num                                             
Amazon_Web_Services 1          1317.985083  21.369638  33.346199  0.694826
                    2          1287.825439  21.049846  32.263959  0.684087
                    3          1291.431372  21.185461  34.601146  0.689070
                    4          1330.254614  21.374129  33.431194  0.694940
                    5          1334.128052  21.328910  31.838752  0.693512
...                                    ...        ...        ...       ...
subway_in_only      5          1299.980762  21.326292  34.060516  0.692803
                    6          1318.748242  21.569754  35.840388  0.700861
                    7          1294.377783  21.331374  35.677078  0.693158
                    8          1314.708276  21.476791  34.027646  0.697776
                    9          1298.002271  21.268003  33.208141  0.690906

[369 rows x 4 columns]

## Acceder à la config originelle d'un hp tuning : 

In [ ]:
import pickle 
import sys
import os
import pandas as pd 

current_path = notebook_dir = os.getcwd()
working_dir = os.path.abspath(os.path.join(current_path, '..','..'))
if working_dir not in sys.path:
    sys.path.insert(0, working_dir)

folder_name = ''
model_args = load_model_args(folder_name)
trial_ids = [#'subway_in_STGCN_MSELoss_2025_01_20_14_27_20569',
             'subway_in_subway_out_STGCN_MSELoss_2025_02_19_00_05_19271',
             'subway_in_subway_out_STGCN_MSELoss_2025_03_29_00_17_68381'
]

for trial_id in trial_ids: 
    print('\nTrial id: ',trial_id)
    args = model_args['model'][f"{trial_id}_f0"]['args']
    print('epochs: ',args['epochs'],' lr: ',args['lr'],' wd: ',args['weight_decay'],' dropout: ',args['dropout'],args['DA_method'])
    if ('scheduler' in args.keys()) and (args['scheduler'] is not None):
        print('Scheduler: milestone: ',args['torch_scheduler_milestone'],
              ' torch_scheduler_gamma: ',args['torch_scheduler_gamma'],
              ' torch_scheduler_lr_start_factor: ',args['torch_scheduler_lr_start_factor'])
    else:
        print('Scheduler: ',None)




Trial id:  subway_in_STGCN_MSELoss_2025_01_20_14_27_20569


KeyError: 'subway_in_STGCN_MSELoss_2025_01_20_14_27_20569_f0'

## Acceder aux train/valid loss d'une K-fold validation :

In [65]:
import sys
import os
import pandas as pd 

current_path = notebook_dir = os.getcwd()
working_dir = os.path.abspath(os.path.join(current_path, '..','..'))
if working_dir not in sys.path:
    sys.path.insert(0, working_dir)
    
from plotting.TS_analysis import plot_TS


save_path = f'save/K_fold_validation/training_with_HP_tuning'
trial_id_100 = 'subway_in_subway_out_STGCN_MSELoss_2025_02_19_00_05_19271'
trial_id_500 = 'subway_in_subway_out_STGCN_MSELoss_2025_03_29_00_17_68381'
trial_id_500_scheduler = 'subway_in_subway_out_STGCN_MSELoss_2025_03_30_03_10_83808'

df_losses100 = pd.read_csv(f"{current_path}/{save_path}/Losses_{trial_id_100}.csv",index_col = 0)
df_losses500 = pd.read_csv(f"{current_path}/{save_path}/Losses_{trial_id_500}.csv",index_col = 0)
df_losses500_scheduler = pd.read_csv(f"{current_path}/{save_path}/Losses_{trial_id_500_scheduler}.csv",index_col = 0)

plot_TS(df_losses100,width=800,height=400,bool_show=True,title=f"Losses on subway-in/subway-out  HP 100 epochs",scatter = False,x_datetime = False)
plot_TS(df_losses500,width=800,height=400,bool_show=True,title=f"Losses on subway-in/subway-out  HP 500 epochs",scatter = False,x_datetime = False)
plot_TS(df_losses500_scheduler,width=800,height=400,bool_show=True,title=f"Losses on subway-in/subway-out HP 500 epochs + scheduler",scatter = False,x_datetime = False)


Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

figure(id='p4877', ...)

## Etude de stabilité des entrainements:

In [6]:
current_path

'/home/rrochas/prediction-validation'

In [18]:
import pickle 
import sys
import os
import pandas as pd 

current_path = notebook_dir = os.getcwd()
working_dir = os.path.abspath(os.path.join(current_path, '..','..'))
if working_dir not in sys.path:
    sys.path.insert(0, working_dir)

save_path = f'save/K_fold_validation/training_with_HP_tuning/tmps'
model_args = pickle.load(open(f'{working_dir}/{save_path}/best_models/model_args.pkl','rb'))
trial_id = 'subway_in_subway_out_STGCN_MSELoss_2025_02_19_00_05_19271'

# Get Loss of each trial, each fold, valid and train: 
df_losses = pd.DataFrame()
for k in range(1,11):
    df_losses_i = pd.read_csv(f"{working_dir}/{save_path}/Losses_{trial_id}_{k}.csv",index_col = 0)
    df_losses_i = df_losses_i.rename(columns = {c:f"trial_{k}_{c}" for c in df_losses_i.columns})
    df_losses = pd.concat([df_losses,df_losses_i],axis=1)
# ...


# Filter only 'valid' loss, plot valid to control stability: 
df_mean_std = pd.DataFrame()
for k in range(5):
    c_valid_trials = [c for c in df_losses.columns if f"f{k}_valid" in c]
    mean_i = df_losses[c_valid_trials].mean(axis=1)
    std_i =  df_losses[c_valid_trials].std(axis=1)
    legend_mean_i,legend_std_i = std_i.values[-1],mean_i.values[-1]

    df_mean_std_fold_k = pd.DataFrame({f"fold{k}_mean":mean_i,
                f"fold{k}_std":std_i,
                })
    df_mean_std = pd.concat([df_mean_std,df_mean_std_fold_k],axis=1)

In [62]:

from bokeh.io import output_notebook, show
from bokeh.plotting import figure,output_file,save
from bokeh.palettes import Category10
from bokeh.models import Legend


# Active l'affichage des graphiques dans le notebook (ou à enlever si utilisation en script)
output_notebook()

p = figure(
    title="Validation loss stability through 10 training",
    x_axis_label="epochs",
    y_axis_label="MSE Loss",
    width = 1000,
)

legend_items = []

# Boucle sur les 5 folds
for k in range(5):
    # Calcul de la légende personnalisée
    legend_mean_i = df_mean_std[f"fold{k}_mean"].iloc[-1]
    legend_std_i  = df_mean_std[f"fold{k}_std"].iloc[-1]
    legend_k = f"\nfold: {k}. Statistics on last value: \nmean = {'{:.2e}'.format(legend_mean_i)}, std = {'{:.2e}'.format(legend_std_i)}\n std/mean : {'{:.2e}'.format(legend_std_i/legend_mean_i)} "
   
    
    # Tracé de la ligne (moyenne)
    line_renderer = p.line(
        x=df_mean_std.index, 
        y=df_mean_std[f"fold{k}_mean"], 
        line_color=Category10[5][k-1]
    )
    
    # Zone +/- std
    varea_renderer = p.varea(
        x=df_mean_std.index,
        y1=df_mean_std[f"fold{k}_mean"] - df_mean_std[f"fold{k}_std"],
        y2=df_mean_std[f"fold{k}_mean"] + df_mean_std[f"fold{k}_std"],
        fill_color=Category10[5][k-1],
        fill_alpha=0.2
    )
    
    # Regroupement pour la légende interactive
    legend_items.append((legend_k, [line_renderer, varea_renderer]))

# Création d'une légende "en dehors" de la zone de tracé
legend = Legend(items=legend_items, location='center')
p.add_layout(legend, 'right')

# Active le click sur la légende pour hide/show les courbes
legend.click_policy = "hide"

show(p)


output_file(f"training_stability_mean_std_validation.html")
save(p)

Loading BokehJS ...

'/home/rrochas/prediction-validation/training_stability_mean_std_validation.html'